In [ ]:
import cv2
import os

# Carpeta con las imágenes originales de la secuencia
images_dir = "./videos/MAX230426_bCatTracking_Exp91_20230426_73604 AM_f0000_t0000.tif"

# Archivo de detecciones generado por predict.py
detections_file = "output/MAX230426_bCatTracking_Exp91_20230426_73604 AM_f0000_t0000.txt"

# Carpeta de salida para imágenes con cuadros
output_dir = "output/visual/3T3-run02"
os.makedirs(output_dir, exist_ok=True)

# Leer detecciones
detections = {}
with open(detections_file) as f:
    for line in f:
        frame_id, obj_id, x, y, w, h, conf, _, _ = map(float, line.strip().split(','))
        frame_id = int(frame_id)
        detections.setdefault(frame_id, []).append((x, y, w, h))

# Dibujar cajas
for frame_id, boxes in detections.items():
    img_path = os.path.join(images_dir, f"{frame_id:06d}.jpg")
    if not os.path.exists(img_path):
        continue
    img = cv2.imread(img_path)
    for x, y, w, h in boxes:
        cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)
    cv2.imwrite(os.path.join(output_dir, f"{frame_id:06d}.jpg"), img)


In [6]:
import cv2
import os
import tifffile as tiff
import numpy as np

# Archivo TIFF con la secuencia de frames
images_file = "./videos/MAX230426_bCatTracking_Exp91_20230426_73604 AM_f0000_t0000.tif"

# Archivo de detecciones generado por predict.py
detections_file = "output/MAX230426_bCatTracking_Exp91_20230426_73604 AM_f0000_t0000.txt"

# Carpeta de salida para imágenes con cuadros
output_dir = "output/visual/imagenes0"
os.makedirs(output_dir, exist_ok=True)

# Leer todos los frames del TIFF
frames_raw = tiff.imread(images_file)
frames = []

for i, f in enumerate(frames_raw):
    # Normalizar a 0-255
    f = ((f - f.min()) / (f.max() - f.min()) * 255).astype(np.uint8)
    
    # Asegurarse de que tenga 3 canales
    if f.ndim == 2:  # gris
        f_bgr = np.stack([f]*3, axis=-1)
    elif f.ndim == 3 and f.shape[2] == 1:  # gris con canal extra
        f_bgr = np.concatenate([f]*3, axis=2)
    elif f.ndim == 3 and f.shape[2] == 3:  # ya BGR o RGB
        f_bgr = f
    else:
        raise ValueError(f"Frame {i} tiene un shape inesperado: {f.shape}")
    
    frames.append(f_bgr)

# Leer detecciones
detections = {}
with open(detections_file) as f:
    for line in f:
        frame_id, obj_id, x, y, w, h, conf, _, _ = map(float, line.strip().split(','))
        frame_id = int(frame_id)
        detections.setdefault(frame_id, []).append((x, y, w, h))

# Dibujar cajas y guardar imágenes
for frame_id, boxes in detections.items():
    if frame_id >= len(frames):
        print(f"Frame {frame_id} no existe en el TIFF")
        continue
    img = frames[frame_id].copy()
    for x, y, w, h in boxes:
        cv2.rectangle(img, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)
    output_path = os.path.join(output_dir, f"{frame_id:06d}.jpg")
    cv2.imwrite(output_path, img)

print("¡Listo! Imágenes guardadas en:", output_dir)


Frame 99 no existe en el TIFF
¡Listo! Imágenes guardadas en: output/visual/imagenes0
